# Data Augmentation

### Note: Run the following code only if you don't have the aug_fer2013 folder. cd to your project repo to make sure the code works.

### The aug_fer2013 folder has already uploaded to my Github. You could use it as your original dataset.

In [ ]:
import cv2
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import matplotlib.image as mpimg
from subprocess import PIPE, Popen
import re

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_dir = 'aug_fer2013/train'
test_dir = 'aug_fer2013/test'

SEED = 12
IMG_HEIGHT = 48
IMG_WIDTH = 48
BATCH_SIZE = 64

In [ ]:
def cmdline(command):
    process = Popen(
        args=command,
        stdout=PIPE,
        shell=True
    )
    string_result = process.communicate()[0]
    string_result = string_result.decode("utf-8") 
    needed = re.findall(r'\d+', string_result)
    return int(needed[0])

In [ ]:
datagen_lst = [ImageDataGenerator(horizontal_flip=True,
                              vertical_flip = False),
           ImageDataGenerator(horizontal_flip = False,
                              vertical_flip = True,
                              zoom_range = 0.1),
           ImageDataGenerator(horizontal_flip = True,
                              vertical_flip = True,
                              height_shift_range=0.1),
           ImageDataGenerator(horizontal_flip = True,
                              vertical_flip = True,
                              height_shift_range=0.1,
                              width_shift_range=0.2),
           ImageDataGenerator(horizontal_flip = False,
                              vertical_flip = True,
                              zoom_range=0.1,
                              height_shift_range=0.1)]

In [ ]:
os.system('mkdir try')
os.system('cp -r fer2013 try/')
os.system('mv try/fer2013 try/aug_fer2013')
train_dir = 'try/aug_fer2013/train'
test_dir = 'try/aug_fer2013/test'

In [ ]:
labels = ['surprise','sad','neutral','happy','fear','disgust','angry']
value_count_labels_trian = [3171, 4830, 4965, 7215, 4097, 436, 3995]
value_count_labels_test = [831, 1247, 1233, 1774, 1024, 111, 958]

for i in range(len(labels)):
    this_label = labels[i]
    this_value_count = value_count_labels_trian[i]
    os.system('mkdir ' + train_dir + '/' + this_label + '_temp')
    os.system('mv ' + train_dir + '/' + this_label + ' ' + train_dir + '/' + this_label + '_temp')
    
    dgl = 1
    #num_emo_image = cmdline("ls " + train_dir + '/' + this_label + '_temp')
    while this_value_count < (7215 //2):
        resized = datagen_lst[dgl].flow_from_directory(train_dir + '/' + this_label + '_temp',
                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                save_to_dir=train_dir + '/' + this_label + '_temp' + '/' + this_label,
                                                color_mode="grayscale", # Choose color mode
                                                class_mode='categorical',
                                                shuffle=True,
                                                save_prefix=str(this_value_count) + '_augmented_',
                                                save_format='jpg', # Formate
                                                batch_size=1)
        for j in range(len(resized)):
            resized.next()
        dgl += 1
        #num_emo_image = cmdline("ls " + train_dir + '/' + this_label + '_temp' + '/' + this_label)
        this_value_count = this_value_count * 2
    
    resized = datagen_lst[0].flow_from_directory(train_dir + '/' + this_label + '_temp',
                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                save_to_dir=train_dir + '/' + this_label + '_temp' + '/' + this_label,
                                                color_mode="grayscale", # Choose color mode
                                                class_mode='categorical',
                                                shuffle=True,
                                                save_prefix=str(this_value_count) + '_augmented_',
                                                save_format='jpg', # Formate
                                                batch_size=1)
    for k in range(7215 - len(resized)):
        resized.next()
        
    os.system('mv ' + train_dir + '/' + this_label + '_temp' + '/' + this_label + ' ' + train_dir)
    os.system('rmdir ' + train_dir + '/' + this_label + '_temp')
    
        